# Data Preprocessing

---
Before we preprocess our data that is going to be used to train and test our machine learning algorithms we first have to ensure our data is consistant and of high quality, so we have to ensure optimal performance. We will have to do the following:

- Handle missing values.
- Encode categorical features.
- Standardize numerical features.

`To view final preprocessing() function, you can scroll down to the bottom`

In [209]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

---

## Missing Values: Categorical

In [210]:
# Load project_adult.csv
df = pd.read_csv('../data/raw/project_adult.csv', index_col=0)  

# Print shape of df
print(f'''
Number of rows: {df.shape[0]}
Number of features: {df.shape[1]}
      ''')

# Check missing values
print(df.isnull().sum())


Number of rows: 26048
Number of features: 15
      
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


In [211]:
# Checking for values in categorical varaibles
df_categorical = df[['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']]

for cat in df_categorical:
    values = df_categorical[cat].unique()
    print(f'''
Unique values for {cat}:
{values}
''')


Unique values for workclass:
['Local-gov' 'Private' 'Self-emp-not-inc' '?' 'Federal-gov' 'Self-emp-inc'
 'State-gov' 'Without-pay' 'Never-worked']


Unique values for education:
['Bachelors' 'Assoc-voc' '9th' 'Some-college' '10th' 'HS-grad'
 'Prof-school' 'Assoc-acdm' '11th' '12th' 'Masters' '7th-8th' 'Doctorate'
 '5th-6th' '1st-4th' 'Preschool']


Unique values for marital-status:
['Never-married' 'Married-civ-spouse' 'Separated' 'Divorced' 'Widowed'
 'Married-spouse-absent' 'Married-AF-spouse']


Unique values for occupation:
['Prof-specialty' 'Exec-managerial' 'Craft-repair' 'Farming-fishing'
 'Other-service' 'Machine-op-inspct' 'Sales' 'Handlers-cleaners'
 'Transport-moving' 'Protective-serv' '?' 'Adm-clerical' 'Priv-house-serv'
 'Tech-support' 'Armed-Forces']


Unique values for relationship:
['Not-in-family' 'Husband' 'Other-relative' 'Unmarried' 'Own-child' 'Wife']


Unique values for race:
['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']


Unique values for 

We have `'?'` for unknown field is the following columns: `workclass`, `occupation`, `native-country`. We will treat this as an unknown value. If the number of rows that contain `'?'` in any of these fields total to around 5-10% of the total dataset we will drop the rows. If its more we will consider imputing instead. 

In [212]:
# Number of rows that contains at lest one field with '?'
num_of_rows = df[df[['workclass','occupation','native-country']].isin(['?']).any(axis=1)].shape[0]

print(f"Number of rows with at least one field with '?': {num_of_rows}")
print(f"Percentage of total rows: {(num_of_rows / df.shape[0]) * 100:.2f}%")

Number of rows with at least one field with '?': 1891
Percentage of total rows: 7.26%


In [213]:
# Drop rows where '?' is present
df = df[~df[['workclass','occupation','native-country']].isin(['?']).any(axis=1)]

---

## Dropping Redundant Columns

We have two features `education` & `education-num`, `education-num` seems to be the numerical representation of `education`. It is standard practive to drop the categorical column to avoid redundancy. We will check if `education` matches `education-num` in each row before dropping `education`.

In [214]:
# Create the mapping dictionary from unique pairs
edu_map = dict(df[['education', 'education-num']].drop_duplicates().values)

# Check if each row's education-num matches the mapping for education
mismatch_mask = df['education-num'] != df['education'].map(edu_map)

# Print rows where there is a mismatch
print(df[mismatch_mask])

Empty DataFrame
Columns: [age, workclass, fnlwgt, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country, income]
Index: []


Every row's categorical education value matches its corresponding numerical value according to our mapping. So we can safely drop the redundant column and keep the mapping for reference. 

In [215]:
# Dropping 'education' column
df = df.drop(columns=['education'])

In [216]:
# Ecuation map for reference
print(edu_map)

{'Bachelors': 13, 'Assoc-voc': 11, '9th': 5, 'Some-college': 10, '10th': 6, 'HS-grad': 9, 'Assoc-acdm': 12, '11th': 7, '12th': 8, 'Masters': 14, '7th-8th': 4, 'Doctorate': 16, 'Prof-school': 15, '5th-6th': 3, '1st-4th': 2, 'Preschool': 1}


---

## Econding Categorical Features

For algorithms like **Perceptron** and **Adaline**, we encode categorical features into numbers because these models compute **weighted sums of inputs**, and they can’t handle raw text or categories—only numeric values that can be multiplied by weights.

In [217]:
# One-hot encoding categorical variables
df = pd.get_dummies(df, columns=[col for col in df_categorical.columns if col != 'education'], dtype=int)

# Verifying one-hot encoding
df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
5514,33,198183,13,0,0,50,>50K,0,1,0,...,0,0,0,0,0,0,0,1,0,0
19777,36,86459,11,0,1887,50,>50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
10781,58,203039,5,0,0,40,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32240,21,180190,11,0,0,46,<=50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
9876,27,279872,10,0,0,40,<=50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0


---

## Standardize Numerical Features: Feature Scaling

Both Perceptron and Adaline are linear models trained with gradient-based updates (or update rules proportional to input values).

- Features are on very different scales (e.g. `age` in years vs. `capital-loss` in dollars), the larger scaled feature dominates the weight updates.
- This can cuase unstable training, slower convergence, or even failure to converge.

Benefits:

- **Perceptron**: scaling helps the algorithm find a cleaner decision boundary because the step updates won’t be skewed toward large-magnitude features.
- **Adaline**: since it minimizes MSE via gradient descent, scaling makes the optimization surface smoother → gradients are balanced → much faster and more stable convergence.

We will use the `StandardScaler()` method from `scikit learn's` `perprocessing` module. 

In [218]:
# Numeric columns
numeric_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Standardizing features
scaler = StandardScaler()

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Verfiying results
df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
5514,-0.413348,0.079327,1.128346,-0.147456,-0.219980,0.763915,>50K,0,1,0,...,0,0,0,0,0,0,0,1,0,0
19777,-0.185241,-0.981089,0.344207,-0.147456,4.442817,0.763915,>50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
10781,1.487546,0.125418,-2.008209,-0.147456,-0.219980,-0.071383,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32240,-1.325777,-0.091451,0.344207,-0.147456,-0.219980,0.429796,<=50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
9876,-0.869562,0.854670,-0.047862,-0.147456,-0.219980,-0.071383,<=50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0


---

## Label Encoding for the Response Variable

Now we will convert the categorical response variable `income` into binary and by doing this we will assing labels to our reponse column.

In [219]:
# Encoding response column
df['income'] = df['income'].map({'>50K': 1, '<=50K': 0})

# Verfiy results
df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
5514,-0.413348,0.079327,1.128346,-0.147456,-0.219980,0.763915,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
19777,-0.185241,-0.981089,0.344207,-0.147456,4.442817,0.763915,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
10781,1.487546,0.125418,-2.008209,-0.147456,-0.219980,-0.071383,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32240,-1.325777,-0.091451,0.344207,-0.147456,-0.219980,0.429796,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
9876,-0.869562,0.854670,-0.047862,-0.147456,-0.219980,-0.071383,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


- `'>50K' → 1` (positive class)
- `'<=50K' → 0` (negative class)

---

## Seperate `X` (Features) & `y` (labels)

The last step will be to seperate the features and labels, so the data can be ready for training. 

In [220]:
# Separate features and target
X = df.drop(columns=['income'], axis=1)
y = df[['income']]

---

## `Preprocess()` Function

Now we will bring everything together and create one method that does everything that we have done until now.

In [221]:
def preprocess_data(df: pd.DataFrame, scaler: StandardScaler | None = None):
    """
    Preprocessing for the Adult Income flat file.

    - Strip strings, turn '?' -> NaN
    - Drop rows with *any* missing values
    - Drop 'education' (keep 'education-num')
    - One-hot encode categoricals
    - Encode target 'income' to {<=50K:0, >50K:1} if present
    - Standardize numeric columns (fit on train if scaler is None; otherwise transform)

    Returns
    -------
    X : DataFrame
    y : Series | None
    scaler : StandardScaler
    """
    # fixed columns
    cat_cols = ['workclass', 'education', 'marital-status', 'occupation',
                'relationship', 'race', 'sex', 'native-country']
    num_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain',
                'capital-loss', 'hours-per-week']

    df = df.copy()
    has_target = 'income' in df.columns

    # 1) clean strings/placeholders
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].map(lambda x: x.strip() if isinstance(x, str) else x)
    df.replace('?', np.nan, inplace=True)

    # 2) drop rows with ANY missing values
    df.dropna(inplace=True)

    # 3) drop redundant education
    if 'education' in df.columns:
        df.drop(columns=['education'], inplace=True, errors='ignore')

    # 4) one-hot encode categoricals
    ohe_cols = [c for c in cat_cols if c != 'education' and c in df.columns]
    if ohe_cols:
        df = pd.get_dummies(df, columns=ohe_cols, dtype=int)

    # 5) split target
    y = None
    if has_target:
        df['income'] = df['income'].map({'>50K': 1, '<=50K': 0})
        y = df['income'].astype('int64')
        X = df.drop(columns=['income'])
    else:
        X = df

    # 6) standardize numerics
    present_num = [c for c in num_cols if c in X.columns]
    fitted_scaler = scaler or StandardScaler()
    if present_num:
        X.loc[:, present_num] = X.loc[:, present_num].astype('float64', copy=False)
        if scaler is None:
            X.loc[:, present_num] = fitted_scaler.fit_transform(X.loc[:, present_num])
        else:
            X.loc[:, present_num] = fitted_scaler.transform(X.loc[:, present_num])

    return X, y, fitted_scaler


We’ll use your `preprocess_data` function to clean the Adult Income dataset, split it into train/test, preprocess with scaling, and then save separate CSVs for features (`X`) and target (`y`). Finally, we’ll show how to preprocess the validation inputs and reload the CSVs back.

- We load the raw dataset `project_adult.csv` and split into train and test sets.
- We stratify by the target (`income`) to preserve class balance
- Fit scaler on training (`scaler=None`).
- Reuse scaler on test (to avoid leakage).
- We save features and labels separately for both train and test.

In [222]:
# Load raw data
df = pd.read_csv("../data/raw/project_adult.csv", index_col=0)

# Split into train and test
train_df, test_df = train_test_split(
    df, test_size=0.3, stratify=df["income"], random_state=42
)

# Preprocess (fit scaler on train, reuse on test)
X_train, y_train, scaler = preprocess_data(train_df, scaler=None)
X_test,  y_test,  _      = preprocess_data(test_df,  scaler=scaler)

# Ensure processed folder exists
processed_path = Path("../data/processed")
processed_path.mkdir(parents=True, exist_ok=True)

# Save features
X_train.to_csv(processed_path / "X_train.csv", index=False)
X_test.to_csv(processed_path / "X_test.csv", index=False)

# Save targets (Series → CSV with single column)
y_train.to_csv(processed_path / "y_train.csv", index=False)
y_test.to_csv(processed_path / "y_test.csv", index=False)

print("Saved X_train, y_train, X_test, y_test to ../data/processed/")

C:\Users\jonat\AppData\Local\Temp\ipykernel_12348\4291609706.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.11277383  0.49183516  0.94670875 ...  0.11277383 -0.79697336
 -1.40347148]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, present_num] = fitted_scaler.fit_transform(X.loc[:, present_num])
C:\Users\jonat\AppData\Local\Temp\ipykernel_12348\4291609706.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.39054222  1.21726734 -1.09376861 ... -1.28862387  0.20965031
 -0.15445564]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, present_num] = fitted_scaler.fit_transform(X.loc[:, present_num])
C:\Users\jonat\AppData\Local\Temp\ipykernel_12348\4291609706.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and w

Saved X_train, y_train, X_test, y_test to ../data/processed/


Validation set has no `income` column, so we just transform using the scaler from training and save features as `X_val.csv`.

In [223]:
# Load validation inputs
val_df = pd.read_csv("../data/raw/project_validation_inputs.csv", index_col=0)

# Preprocess using the same scaler from training
X_val, _, _ = preprocess_data(val_df, scaler=scaler)

# Save features only
X_val.to_csv(processed_path / "X_val.csv", index=False)

print("Saved X_val.csv to ../data/processed/")

Saved X_val.csv to ../data/processed/


C:\Users\jonat\AppData\Local\Temp\ipykernel_12348\4291609706.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.87278562  0.49183516 -0.72116109 ...  0.64345969 -0.87278562
 -1.55509601]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, present_num] = fitted_scaler.transform(X.loc[:, present_num])
C:\Users\jonat\AppData\Local\Temp\ipykernel_12348\4291609706.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.28225217 -1.31635117 -0.03950858 ...  1.54771184 -1.3815891
 -1.5496605 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, present_num] = fitted_scaler.transform(X.loc[:, present_num])
C:\Users\jonat\AppData\Local\Temp\ipykernel_12348\4291609706.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

---

## Reload Processed CSVs for Training

In [224]:
# Reload features
X_train = pd.read_csv("../data/processed/X_train.csv")
X_test  = pd.read_csv("../data/processed/X_test.csv")

# Reload targets (squeezed into Series)
y_train = pd.read_csv("../data/processed/y_train.csv").squeeze("columns")
y_test  = pd.read_csv("../data/processed/y_test.csv").squeeze("columns")

# Reload validation features
X_val = pd.read_csv("../data/processed/X_val.csv")